# Deep Learning for Business Applications course

## TOPIC 7: Face Recognition with DeepFace

### 1. Library installation

[DeepFace](https://github.com/serengil/deepface?tab=readme-ov-filehttps://github.com/serengil/deepface?tab=readme-ov-file) is a lightweight face recognition and facial attribute analysis (age, gender, emotion and race) framework for python.

In [ ]:
!pip install deepface
!pip install opencv-python

In [ ]:
import os
import cv2
import requests
from PIL import Image
import numpy as np
from deepface import DeepFace
from matplotlib import pyplot as plt

### 2. Embeddings

Let's use a [picture of a teacher](https://gsom.spbu.ru/about-gsom/faculty/garshin/) from GSOM site.

In [ ]:
img_url = 'https://gsom.spbu.ru/images/1faces/garshin.jpg'
img = Image.open(
    requests.get(img_url, stream=True).raw
).convert('RGB')
plt.figure(figsize=(4, 4))
plt.imshow(img)
plt.show()

In [ ]:
!mkdir -p db

In [ ]:
img.save('db/vgarshin.jpg')

In [ ]:
def dfproc(img_path, fsize=(6, 6), model_name='VGG-Face'):
    # load image
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    print('image loaded:', img_path)

    # plot an image loaded
    if fsize:
        plt.figure(figsize=fsize)
        plt.imshow(img)
        plt.show()

    # process with DeepFace
    embedding_objs = DeepFace.represent(
        img_path,
        model_name=model_name
    )

    # draw resuls
    for i, emb in enumerate(embedding_objs):
        # To draw a rectangle in a face
        face = emb['facial_area']
        x, y, w, h = face['x'], face['y'], face['w'], face['h']
        cv2.rectangle(
            img,
            (x, y),
            (x + w, y + h),
            (0, 255, 0),
            2
        )
        descr = f'face {i}, conf={emb["face_confidence"]}'
        cv2.putText(
            img,
            descr,
            (x, y),
            cv2.FONT_HERSHEY_SIMPLEX,
            1.25,
            (0, 255, 0),
            3
        )
    if fsize:
        plt.figure(figsize=fsize)
        plt.imshow(img)
        plt.show()

    return embedding_objs

In [ ]:
embedding_objs = dfproc(img_path='db/vgarshin.jpg')

In [ ]:
embedding_objs = dfproc(img_path='imgs/faces1.jpg', fsize=(8, 8))

In [ ]:
len(embedding_objs)

In [ ]:
embedding_objs[0].keys()

In [ ]:
embedding_objs[0]['facial_area']

In [ ]:
embedding_objs[0]['face_confidence']

In [ ]:
len(embedding_objs[0]['embedding'])

### 3. Advanced analysis

In [ ]:
def dfanalyze(img_path, fsize=(6, 6)):
    # load image
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    print('image loaded:', img_path)

    # plot an image loaded
    if fsize:
        plt.figure(figsize=fsize)
        plt.imshow(img)
        plt.show()

    # process with DeepFace
    results = DeepFace.analyze(img_path)

    # draw resuls
    for i, res in enumerate(results):
        # To draw a rectangle in a face
        face = res['region']
        x, y, w, h = face['x'], face['y'], face['w'], face['h']
        cv2.rectangle(
            img,
            (x, y),
            (x + w, y + h),
            (0, 255, 0),
            2
        )
        emo = max(res['gender'], key=res['gender'].get)
        descr = ', '.join([
            emo,
            res['dominant_emotion'],
            'age ' + str(res['age'])
        ])
        cv2.putText(
            img,
            descr,
            (x, y),
            cv2.FONT_HERSHEY_SIMPLEX,
            1.25,
            (0, 255, 0),
            3
        )
    if fsize:
        plt.figure(figsize=fsize)
        plt.imshow(img)
        plt.show()

    return results

In [ ]:
results = dfanalyze(img_path='db/vgarshin.jpg')

In [ ]:
results = dfanalyze(img_path='imgs/faces1.jpg', fsize=(8, 8))

In [ ]:
results[0]

### 4. Face recognition

#### 4.1. Manual matching

In [ ]:
MODELS = [
    'VGG-Face',
    'Facenet',
    'Facenet512',
    'OpenFace',
    'DeepFace',
    'DeepID',
    'ArcFace',
    'Dlib',
    'SFace',
    'GhostFaceNet'
]

In [ ]:
def cosine_sim(x, y):
    return np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))

In [ ]:
emb_vgarshin = dfproc(
    img_path='db/vgarshin.jpg',
    fsize=None,
    model_name=MODELS[2]
)
emb_friends = dfproc(
    img_path='imgs/faces1.jpg',
    fsize=None,
    model_name=MODELS[2]
)

In [ ]:
for i, emb in enumerate(emb_friends):
    cos_sim = cosine_sim(
        emb_vgarshin[0]['embedding'],
        emb['embedding']
    )
    print(f'vgarshin and face {i} =', cos_sim)

#### 4.2. Built-in tools

In [ ]:
img_url = 'https://gsom.spbu.ru/images/1faces/gorovoi.jpg'
img = Image.open(
    requests.get(img_url, stream=True).raw
).convert('RGB')
img.save('db/vgorovoi.jpg')
plt.figure(figsize=(4, 4))
plt.imshow(img)
plt.show()

In [ ]:
img_url = 'https://gsom.spbu.ru/images/1faces/bova.jpg'
img = Image.open(
    requests.get(img_url, stream=True).raw
).convert('RGB')
img.save('db/sbova.jpg')
plt.figure(figsize=(4, 4))
plt.imshow(img)
plt.show()

In [ ]:
# face verification

result = DeepFace.verify(
  img1_path="db/sbova.jpg",
  img2_path="db/vgarshin.jpg",
  model_name=MODELS[0]
)
print(result)

In [ ]:
img_path = 'imgs/faces1.jpg'
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(6, 6))
plt.imshow(img)
plt.show()

In [ ]:
# take face 1 from friends image

emb_friends[1]['facial_area']

In [ ]:
face = emb_friends[1]['facial_area']
x, y, w, h = face['x'], face['y'], face['w'], face['h']
img_vgarshin = img[y:y + h, x:x + w, :]
plt.figure(figsize=(2, 2))
plt.imshow(img_vgarshin)
plt.show()

In [ ]:
# face verification

result = DeepFace.verify(
  img1_path=img_vgarshin,
  img2_path='db/vgarshin.jpg',
  model_name=MODELS[0]
)
print(result)

In [ ]:
# face recognition

results = DeepFace.find(
  img_path=img_vgarshin,  # face to find
  db_path='db',  # path to directory with faces
  model_name=MODELS[0]
)

In [ ]:
type(results)

In [ ]:
len(results)

In [ ]:
results[0]

In [ ]:
!ls -la db/

In [ ]:
!rm -rf db/.ipynb_checkpoints

### 5. Clean up

In [ ]:
!ls -la ~/.deepface

In [ ]:
!rm -rf ~/.deepface/weights

### <font color='red'>HOME ASSIGNMENT (Option #1)</font>

Your goal is to make your own database of faces and test `DeepFace.verify()` and `DeepFace.find()` methods. You may use your own photos or photos of the celebrities (in a case you do not want use your face for home assignment).

Please collect at least 5 images for database.